# Samples

In [1]:
hf_dataset = 'paperswithcode_aspects'
nlp_cache_dir = './data/nlp_cache'

index_dir = './output/pwc/whoosh_index'
tensorboard_dir = './output/pwc/tensorboard'

# SPECTER
general_fp = './output/pwc/specter.w2v.txt'  

# Fine-tuned Sentence-SciBERT
task_fp = './output/pwc/task/1/st_scibert-scivocab-uncased/pwc_id2vec__all_docs.w2v.txt'  # pwc_id2vec__train_and_test.w2v.txt  
method_fp = './output/pwc/method/1/st_scibert-scivocab-uncased/pwc_id2vec__all_docs.w2v.txt' 
dataset_fp = './output/pwc/dataset/1/st_scibert-scivocab-uncased/pwc_id2vec__all_docs.w2v.txt' 


In [2]:
sample_ids = [
    'V9yaFJy03j', # Data augmentation for low resource sentiment analysis using generative adversarial networks
    '9rB_3A5Wy0',  # Sentiment Analysis of German Twitter
    'N0jcCtrbqO',  # Fake News in Social Networks
]

In [3]:
from gensim.models import KeyedVectors
import os
import random
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
from datasets import load_dataset
from experiments.utils import get_local_hf_dataset_path
from fuzzywuzzy import fuzz, process
from tqdm.auto import tqdm
from paperswithcode import Paper
from IPython.core.display import display, HTML

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
general_vecs = KeyedVectors.load_word2vec_format(general_fp)
task_vecs = KeyedVectors.load_word2vec_format(task_fp)
method_vecs = KeyedVectors.load_word2vec_format(method_fp)
dataset_vecs = KeyedVectors.load_word2vec_format(dataset_fp)

# Normalize vectors
general_vecs.init_sims(replace=True)
task_vecs.init_sims(replace=True)
method_vecs.init_sims(replace=True)
dataset_vecs.init_sims(replace=True)

In [5]:
# Load meta data
docs_ds = load_dataset(get_local_hf_dataset_path(hf_dataset),
                       name='docs',
                       cache_dir=nlp_cache_dir,
                          split='docs')

Reusing dataset pwc_aspects (./data/nlp_cache/pwc_aspects/docs/0.1.0/5a02274a50cfdd54f404ef512e5453e7f1a9db1cef85c4ae2ecd4607bc43943e)


In [6]:
paper_id2paper = {p['paper_id']: Paper(**p) for p in docs_ds}

In [7]:
# Find neighbors
max_abstract_length = 200
top_k = 5

def get_html(rank, paper, score=0., show_details=True):  
    details_html = f'''
        <p style="margin: 0; padding: 0; font-size: 1.1rem">
        <b>Tasks:</b> {", ".join(paper.aspect_tasks)}
        <b>Methods:</b> {", ".join(paper.aspect_methods)}
        <b>Datasets:</b> {", ".join(paper.aspect_datasets)}
        </p><p>
        {paper.abstract[:max_abstract_length]} <a href="#" onclick="$('#abstract_{paper.paper_id}').toggle();">...</a>
        <span id="abstract_{paper.paper_id}" style="display:none">{paper.abstract[max_abstract_length:]}</span>
        </div>
        ''' if show_details else ''
        
    return f'''
        <div>
        <h3>{rank}. <a href="{paper.paper_url}">{paper.title}</a> <small style="background: #eee">{score:.3f}</small></h3>
        {details_html}
        </div>
        '''
    

def get_col(nn, rank, show_details=True):
    pid, score = nn[rank - 1]
    return f'<td style="vertical-align: top">{get_html(rank, paper_id2paper[pid], score, show_details)}</td>'

def get_multi_view_html(seed_id, show_details=True):

    general_nn = list(general_vecs.most_similar(seed_id, topn=top_k))
    task_nn = list(task_vecs.most_similar(seed_id, topn=top_k))
    method_nn = list(method_vecs.most_similar(seed_id, topn=top_k))
    dataset_nn = list(dataset_vecs.most_similar(seed_id, topn=top_k))
    col_width = '25%'

    results_html = get_html('Seed', paper_id2paper[seed_id]) + ' <hr />'
    
    results_html += f'''
    <table><thead><tr>
    <th width="{col_width}">General purpose</th>
    <th width="{col_width}">Task-related</th>
    <th width="{col_width}">Method-related</th>
    <th width="{col_width}">Dataset-related</th>
    </thead><tbody>
    '''

    for rank in range(1, top_k+1):
        results_html += f'<tr style="vertical-align: top">'
        results_html += get_col(general_nn, rank, show_details)
        results_html += get_col(task_nn, rank, show_details)
        results_html += get_col(method_nn, rank, show_details)
        results_html += get_col(dataset_nn, rank, show_details)

        results_html += f'</tr>'

        #break

    results_html += '</tbody></table>'

    return results_html


## Select samples

In [8]:
for pid in sample_ids:
    display(HTML(get_multi_view_html(pid, show_details=True)))

General purpose,Task-related,Method-related,Dataset-related
"1. Adversarial Training for Aspect-Based Sentiment Analysis with BERT 0.846 Tasks: Aspect-Based Sentiment Analysis, Aspect Extraction, Language Modelling, Sentiment Analysis Methods: Residual Connection, Attention Dropout, Linear Warmup With Linear Decay, Weight Decay, GELU, Dense Connections, Adam, WordPiece, Softmax, Dropout, Multi-Head Attention, Layer Normalization, Scaled Dot-Product Attention, BERT Datasets: SemEval 2014 Task 4 Sub Task 2 Aspect-Based Sentiment Analysis (ABSA) deals with the extraction of sentiments and their targets. Collecting labeled data for this task in order to help neural networks generalize better can be labori ... ous and time-consuming. As an alternative, similar data to the real-world examples can be produced artificially through an adversarial process which is carried out in the embedding space. Although these examples are not real sentences, they have been shown to act as a regularization method which can make neural networks more robust. In this work, we apply adversarial training, which was put forward by Goodfellow et al. (2014), to the post-trained BERT (BERT-PT) language model proposed by Xu et al. (2019) on the two major tasks of Aspect Extraction and Aspect Sentiment Classification in sentiment analysis. After improving the results of post-trained BERT by an ablation study, we propose a novel architecture called BERT Adversarial Training (BAT) to utilize adversarial training in ABSA. The proposed model outperforms post-trained BERT in both tasks. To the best of our knowledge, this is the first study on the application of adversarial training in ABSA.","1. Not Enough Data? Deep Learning to the Rescue! 0.987 Tasks: Data Augmentation, Language Modelling, Text Classification, Text Generation Methods: Datasets: Based on recent advances in natural language modeling and those in text generation capabilities, we propose a novel data augmentation method for text classification tasks. We use a powerful pre-traine ... d neural network model to artificially synthesize new labeled data for supervised learning. We mainly focus on cases with scarce labeled data. Our method, referred to as language-model-based data augmentation (LAMBADA), involves fine-tuning a state-of-the-art language generator to a specific task through an initial training phase on the existing (usually small) labeled data. Using the fine-tuned model and given a class label, new sentences for the class are generated. Our process then filters these new sentences by using a classifier trained on the original data. In a series of experiments, we show that LAMBADA improves classifiers' performance on a variety of datasets. Moreover, LAMBADA significantly improves upon the state-of-the-art techniques for data augmentation, specifically those applicable to text classification tasks with little data.","1. DNA Methylation Data to Predict Suicidal and Non-Suicidal Deaths: A Machine Learning Approach 0.980 Tasks: Data Visualization, Dimensionality Reduction Methods: PCA Datasets: The objective of this study is to predict suicidal and non-suicidal deaths from DNA methylation data using a modern machine learning algorithm. We used support vector machines to classify existing sec ... ondary data consisting of normalized values of methylated DNA probe intensities from tissues of two cortical brain regions to distinguish suicide cases from control cases. Before classification, we employed Principal component analysis (PCA) and t-distributed Stochastic Neighbor Embedding (t-SNE) to reduce the dimension of the data. In comparison to PCA, the modern data visualization method t-SNE performs better in dimensionality reduction. t-SNE accounts for the possible non-linear patterns in low-dimensional data. We applied four-fold cross-validation in which the resulting output from t-SNE was used as training data for the Support Vector Machine (SVM). Despite the use of cross-validation, the nominal

General purpose,Task-related,Method-related,Dataset-related
"1. Comparing methods for Twitter Sentiment Analysis 0.873 Tasks: Sentiment Analysis, Twitter Sentiment Analysis Methods: SVM, Logistic Regression Datasets: This work extends the set of works which deal with the popular problem of sentiment analysis in Twitter. It investigates the most popular document (""tweet"") representation methods which feed sentiment ... evaluation mechanisms. In particular, we study the bag-of-words, n-grams and n-gram graphs approaches and for each of them we evaluate the performance of a lexicon-based and 7 learning-based classification algorithms (namely SVM, Na\""ive Bayesian Networks, Logistic Regression, Multilayer Perceptrons, Best-First Trees, Functional Trees and C4.5) as well as their combinations, using a set of 4451 manually annotated tweets. The results demonstrate the superiority of learning-based methods and in particular of n-gram graphs approaches for predicting the sentiment of tweets. They also show that the combinatory approach has impressive effects on n-grams, raising the confidence up to 83.15% on the 5-Grams, using majority vote and a balanced dataset (equal number of positive, negative and neutral tweets for training). In the n-gram graph cases the improvement was small to none, reaching 94.52% on the 4-gram graphs, using Orthodromic distance and a threshold of 0.001.","1. Opinion Mining on Non-English Short Text 0.993 Tasks: Opinion Mining, Sentiment Analysis Methods: Datasets: As the type and the number of such venues increase, automated analysis of sentiment on textual resources has become an essential data mining task. In this paper, we investigate the problem of mining o ... pinions on the collection of informal short texts. Both positive and negative sentiment strength of texts are detected. We focus on a non-English language that has few resources for text mining. This approach would help enhance the sentiment analysis in languages where a list of opinionated words does not exist. We propose a new method projects the text into dense and low dimensional feature vectors according to the sentiment strength of the words. We detect the mixture of positive and negative sentiments on a multi-variant scale. Empirical evaluation of the proposed framework on Turkish tweets shows that our approach gets good results for opinion mining.","1. VICTOR: a Dataset for Brazilian Legal Documents Classification 0.986 Tasks: Methods: Datasets: BVICTOR, MVICTOR (theme), SVICTOR (theme), MVICTOR (type), SVICTOR (type) This paper describes VICTOR, a novel dataset built from Brazil{'}s Supreme Court digitalized legal documents, composed of more than 45 thousand appeals, which includes roughly 692 thousand documents{- ... --}about 4.6 million pages. The dataset contains labeled text data and supports two types of tasks: document type classification; and theme assignment, a multilabel problem. We present baseline results using bag-of-words models, convolutional neural networks, recurrent neural networks and boosting algorithms. We also experiment using linear-chain Conditional Random Fields to leverage the sequential nature of the lawsuits, which we find to lead to improvements on document type classification. Finally we compare a theme classification approach where we use domain knowledge to filter out the less informative document pages to the default one where we use all pages. Contrary to the Court experts{'} expectations, we find that using all available data is the better method. We make the dataset available in three versions of different sizes and contents to encourage explorations of better models and techniques.","1. A Simple Approach to Multilingual Polarity Classification in Twitter 0.988 Tasks: Sentiment Analysis Methods: Datasets: Recently, sentiment analysis has received a lot of attention due to the interest in mining opinions of social media users. Sentiment analysis consists in determining the polarity of a given text, i.

General purpose,Task-related,Method-related,Dataset-related
"1. Lean From Thy Neighbor: Stochastic & Adversarial Bandits in a Network 0.791 Tasks: Decision Making Methods: Datasets: An individual's decisions are often guided by those of his or her peers, i.e., neighbors in a social network. Presumably, being privy to the experiences of others aids in learning and decision making, ... but how much advantage does an individual gain by observing her neighbors? Such problems make appearances in sociology and economics and, in this paper, we present a novel model to capture such decision-making processes and appeal to the classical multi-armed bandit framework to analyze it. Each individual, in addition to her own actions, can observe the actions and rewards obtained by her neighbors, and can use all of this information in order to minimize her own regret. We provide algorithms for this setting, both for stochastic and adversarial bandits, and show that their regret smoothly interpolates between the regret in the classical bandit setting and that of the full-information setting as a function of the neighbors' exploration. In the stochastic setting the additional information must simply be incorporated into the usual estimation of the rewards, while in the adversarial setting this is attained by constructing a new unbiased estimator for the rewards and appropriately bounding the amount of additional information provided by the neighbors. These algorithms are optimal up to log factors; despite the fact that the agents act independently and selfishly, this implies that it is an approximate Nash equilibria for all agents to use our algorithms. Further, we show via empirical simulations that our algorithms, often significantly, outperform existing algorithms that one could apply to this setting.","1. Defending Against Neural Fake News 0.998 Tasks: Fake News Detection, Text Generation Methods: Datasets: Grover-Mega Recent progress in natural language generation has raised dual-use concerns. While applications like summarization and translation are positive, the underlying technology also might enable adversaries ... to generate neural fake news: targeted propaganda that closely mimics the style of real news. Modern computer security relies on careful threat modeling: identifying potential threats and vulnerabilities from an adversary's point of view, and exploring potential mitigations to these threats. Likewise, developing robust defenses against neural fake news requires us first to carefully investigate and characterize the risks of these models. We thus present a model for controllable text generation called Grover. Given a headline like `Link Found Between Vaccines and Autism,' Grover can generate the rest of the article; humans find these generations to be more trustworthy than human-written disinformation. Developing robust verification techniques against generators like Grover is critical. We find that best current discriminators can classify neural fake news from real, human-written, news with 73% accuracy, assuming access to a moderate level of training data. Counterintuitively, the best defense against Grover turns out to be Grover itself, with 92% accuracy, demonstrating the importance of public release of strong generators. We investigate these results further, showing that exposure bias -- and sampling strategies that alleviate its effects -- both leave artifacts that similar discriminators can pick up on. We conclude by discussing ethical issues regarding the technology, and plan to release Grover publicly, helping pave the way for better detection of neural fake news.","1. Sim-to-Real Transfer Learning using Robustified Controllers in Robotic Tasks involving Complex Dynamics 0.986 Tasks: Transfer Learning Methods: Datasets: Learning robot tasks or controllers using deep reinforcement learning has been proven effective in simulations. Learning in simulation has several advantages. For example, one can fully control the si 

## Random samples

In [10]:
random_ids = random.sample(list(paper_id2paper.keys()), 3)

for pid in random_ids:
    display(HTML(get_multi_view_html(pid, show_details=True)))

General purpose,Task-related,Method-related,Dataset-related
"1. Bank distress in the news: Describing events through deep learning 0.792 Tasks: Methods: Datasets: While many models are purposed for detecting the occurrence of significant events in financial systems, the task of providing qualitative detail on the developments is not usually as well automated. W ... e present a deep learning approach for detecting relevant discussion in text and extracting natural language descriptions of events. Supervised by only a small set of event information, comprising entity names and dates, the model is leveraged by unsupervised learning of semantic vector representations on extensive text data. We demonstrate applicability to the study of financial risk based on news (6.6M articles), particularly bank distress and government interventions (243 events), where indices can signal the level of bank-stress-related reporting at the entity level, or aggregated at national or European level, while being coupled with explanations. Thus, we exemplify how text, as timely, widely available and descriptive data, can serve as a useful complementary source of information for financial and systemic risk analytics.","1. Bank distress in the news: Describing events through deep learning 0.982 Tasks: Methods: Datasets: While many models are purposed for detecting the occurrence of significant events in financial systems, the task of providing qualitative detail on the developments is not usually as well automated. W ... e present a deep learning approach for detecting relevant discussion in text and extracting natural language descriptions of events. Supervised by only a small set of event information, comprising entity names and dates, the model is leveraged by unsupervised learning of semantic vector representations on extensive text data. We demonstrate applicability to the study of financial risk based on news (6.6M articles), particularly bank distress and government interventions (243 events), where indices can signal the level of bank-stress-related reporting at the entity level, or aggregated at national or European level, while being coupled with explanations. Thus, we exemplify how text, as timely, widely available and descriptive data, can serve as a useful complementary source of information for financial and systemic risk analytics.","1. Jointly Learning to Detect Emotions and Predict Facebook Reactions 0.978 Tasks: Emotion Classification Methods: Datasets: The growing ubiquity of Social Media data offers an attractive perspective for improving the quality of machine learning-based models in several fields, ranging from Computer Vision to Natural Languag ... e Processing. In this paper we focus on Facebook posts paired with reactions of multiple users, and we investigate their relationships with classes of emotions that are typically considered in the task of emotion detection. We are inspired by the idea of introducing a connection between reactions and emotions by means of First-Order Logic formulas, and we propose an end-to-end neural model that is able to jointly learn to detect emotions and predict Facebook reactions in a multi-task environment, where the logic formulas are converted into polynomial constraints. Our model is trained using a large collection of unsupervised texts together with data labeled with emotion classes and Facebook posts that include reactions. An extended experimental analysis that leverages a large collection of Facebook posts shows that the tasks of emotion classification and reaction prediction can both benefit from their interaction.","1. Deep learning bank distress from news and numerical financial data 0.979 Tasks: Methods: Datasets: In this paper we focus our attention on the exploitation of the information contained in financial news to enhance the performance of a classifier of bank distress. Such information should be analyzed ... and inserted into the predictive model in the most efficient way and th

General purpose,Task-related,Method-related,Dataset-related
"1. Guided Dyna-Q for Mobile Robot Exploration and Navigation 0.811 Tasks: Methods: Datasets: Model-based reinforcement learning (RL) enables an agent to learn world models from trial-and-error experiences toward achieving long-term goals. Automated planning, on the other hand, can be used for ... accomplishing tasks through reasoning with declarative action knowledge. Despite their shared goal of completing complex tasks, the development of RL and automated planning has mainly been isolated due to their different modalities of computation. Focusing on improving model-based RL agent's exploration strategy and sample efficiency, we develop Guided Dyna-Q (GDQ) to enable RL agents to reason with action knowledge to avoid exploring less-relevant states toward more efficient task accomplishment. GDQ has been evaluated in simulation and using a mobile robot conducting navigation tasks in an office environment. Results show that GDQ reduces the effort in exploration while improving the quality of learned policies.","1. Interactive Gibson Benchmark: A Benchmark for Interactive Navigation in Cluttered Environments 0.997 Tasks: Robot Navigation Methods: Datasets: We present Interactive Gibson Benchmark, the first comprehensive benchmark for training and evaluating Interactive Navigation: robot navigation strategies where physical interaction with objects is al ... lowed and even encouraged to accomplish a task. For example, the robot can move objects if needed in order to clear a path leading to the goal location. Our benchmark comprises two novel elements: 1) a new experimental setup, the Interactive Gibson Environment, which simulates high fidelity visuals of indoor scenes, and high fidelity physical dynamics of the robot and common objects found in these scenes; 2) a set of Interactive Navigation metrics which allows one to study the interplay between navigation and physical interaction. We present and evaluate multiple learning-based baselines in Interactive Gibson, and provide insights into regimes of navigation with different trade-offs between navigation path efficiency and disturbance of surrounding objects. We make our benchmark publicly available(https://sites.google.com/view/interactivegibsonenv) and encourage researchers from all disciplines in robotics (e.g. planning, learning, control) to propose, evaluate, and compare their Interactive Navigation solutions in Interactive Gibson.","1. TPO: TREE SEARCH POLICY OPTIMIZATION FOR CONTINUOUS ACTION SPACES 0.989 Tasks: Methods: Entropy Regularization, PPO Datasets: Monte Carlo Tree Search (MCTS) has achieved impressive results on a range of discrete environments, such as Go, Mario and Arcade games, but it has not yet fulfilled its true potential in continuous do ... mains.In this work, we introduceTPO, a tree search based policy optimization method for continuous environments. TPO takes a hybrid approach to policy optimization. Building the MCTS tree in a continuous action space and updating the policy gradient using off-policy MCTS trajectories are non-trivial. To overcome these challenges, we propose limiting tree search branching factor by drawing only few action samples from the policy distribution and define a new loss function based on the trajectories’ mean and standard deviations. Our approach led to some non-intuitive findings. MCTS training generally requires a large number of samples and simulations. However, we observed that bootstrappingtree search with a pre-trained policy allows us to achieve high quality results with a low MCTS branching factor and few number of simulations. Without the proposed policy bootstrapping, continuous MCTS would require a much larger branching factor and simulation count, rendering it computationally and prohibitively expensive. In our experiments, we use PPO as our baseline policy optimization algorithm. TPO significantly improves the policy on nearly all of our benchmarks. For

General purpose,Task-related,Method-related,Dataset-related
"1. Convolutional LSTM Network: A Machine Learning Approach for Precipitation Nowcasting 0.852 Tasks: Weather Forecasting Methods: Convolution, ConvLSTM, Sigmoid Activation, Tanh Activation, LSTM Datasets: The goal of precipitation nowcasting is to predict the future rainfall intensity in a local region over a relatively short period of time. Very few previous studies have examined this crucial and chal ... lenging weather forecasting problem from the machine learning perspective. In this paper, we formulate precipitation nowcasting as a spatiotemporal sequence forecasting problem in which both the input and the prediction target are spatiotemporal sequences. By extending the fully connected LSTM (FC-LSTM) to have convolutional structures in both the input-to-state and state-to-state transitions, we propose the convolutional LSTM (ConvLSTM) and use it to build an end-to-end trainable model for the precipitation nowcasting problem. Experiments show that our ConvLSTM network captures spatiotemporal correlations better and consistently outperforms FC-LSTM and the state-of-the-art operational ROVER algorithm for precipitation nowcasting.","1. Statistical post-processing of wind speed forecasts using convolutional neural networks 0.998 Tasks: Weather Forecasting Methods: Datasets: Current statistical post-processing methods for probabilistic weather forecasting are not capable of using full spatial patterns from the numerical weather prediction (NWP) model. In this paper we inc ... orporate spatial wind speed information by using convolutional neural networks (CNNs) and obtain probabilistic wind speed forecasts in the Netherlands for 48 hours ahead, based on KNMI's Harmonie-Arome NWP model. The CNNs are shown to have higher Brier skill scores for medium to higher wind speeds, as well as a better continuous ranked probability score (CRPS), than fully connected neural networks and quantile regression forests.","1. Statistical Downscaling of Temperature Distributions from the Synoptic Scale to the Mesoscale Using Deep Convolutional Neural Networks 0.980 Tasks: Methods: Datasets: Deep learning, particularly convolutional neural networks for image recognition, has been recently used in meteorology. One of the promising applications is developing a statistical surrogate model th ... at converts the output images of low-resolution dynamic models to high-resolution images. Our study exhibits a preliminary experiment that evaluates the performance of a model that downscales synoptic temperature fields to mesoscale temperature fields every 6 hours. The deep learning model was trained with operational 22-km gridded global analysis surface winds and temperatures as the input, operational 5-km gridded regional analysis surface temperatures as the desired output, and a target domain covering central Japan. The results confirm that our deep convolutional neural network (DCNN) is capable of estimating the locations of coastlines and mountain ridges in great detail, which are not retained in the inputs, and providing high-resolution surface temperature distributions. For instance, while the average root-mean-square error (RMSE) is 2.7 K between the global and regional analyses at altitudes greater than 1000 m, the RMSE is reduced to 1.0 K, and the correlation coefficient is improved from 0.6 to 0.9 by the surrogate model. Although this study evaluates a surrogate model only for surface temperature, it probably can be improved by augmenting the downscaling variables and vertical profiles. Surrogate models of DCNNs require only a small amount of computational power once their training is finished. Therefore, if the surrogate models are implemented at short time intervals, they will provide high-resolution weather forecast guidance or environment emergency alerts at low cost.","1. A framework for probabilistic weather forecast post-processing across models and lead times using machine learnin